In [196]:
# Dependencies and Setup
import json
import requests
import pandas as pd

In [197]:
# List of years
year_list = ["2014", "2015", "2016", "2017", "2018"]

# Set url for API
url = "https://api.bjs.ojp.gov/bjs/ncvs/v2/personal/"

# Set empty lists to hold personal data
weight = []
ager = []
gender = []
hincome = []
newcrime = []
locationr = []
years = []
region=[]
notify=[]

# Loop through each year
for year in year_list:

    # Create search query, make a request and store in JSON file
    query = url + year + "?format=json" 
    response = requests.get(query)
    response_json = response.json()
    
    data_list=response_json["personalData"]
  
    for data in data_list:
        
        try: 
            region.append(data['region'])
            years.append(data['year'])
            weight.append(data['weight'])
            ager.append(data['ager'])
            notify.append(data['notify'])
            gender.append(data['gender'])
            hincome.append(data['hincome'])
            newcrime.append(data['newoff'])
            locationr.append(data['locationr'])
        except Exception:
            print(data)

In [198]:
# Create a dataframe
ncvs = pd.DataFrame({
    "region":region,
    "file report":notify,
    "year": years,
    "weight": weight,
    "ager": ager,
    "gender": gender,
    "hincome": hincome,
    "newcrime":newcrime,
    "locationr": locationr})


In [199]:
# Get a list of all of our columns for easy reference for PERSONAL
ncvs.head()

,region,file report,year,weight,ager,gender,hincome,newcrime,locationr
0,3,1,2014,2877.2646,5,2,4,1,2
1,3,2,2014,4079.9336,4,1,88,3,3
2,3,1,2014,5984.119,4,2,88,3,2
3,3,2,2014,3271.8376,6,1,4,3,5
4,3,1,2014,3402.2559,6,1,88,4,1


In [200]:
# Remove the rows with missing data
#ncvs_df = ncvs_df.dropna(how = "any")
ncvs['ager'] = ncvs['ager'].replace({"1":"12-14",
        "2":"15-17",
        "3": "18-20",
        "4": "21-24",                         
        "5": "25-34",
        "6": "35-49",
        "7": "50-64",                         
        "8": "65+"})

ncvs['gender'] = ncvs['gender'].replace({"1":"Male",
        "2":"Female"})


ncvs['region'] = ncvs['region'].replace({"1":"NorthEast",
        "2": "MidWest",
        "3": "South",
        "4": "West"})

ncvs['file report'] = ncvs['file report'].replace({"1":"Yes",
        "2": "No",
        "3": "Do not know"})



ncvs['hincome'] = ncvs['hincome'].replace({"1":"<7.5k",
        "2": "7.5k-14.9k",
        "3": "15k-24.9k",
        "4": "25k-34.9k",
        "5": "35k-49.9k",
        "6": "50k-74.9k",
        "7": "75k+",
        "88": "Unknown"})

ncvs['locationr'] = ncvs['locationr'].replace({"1":"victim's home",
        "2": " friend/neighbor/relative's home",
        "3": "com/parking lot or public",
        "4": "school",
        "5": "other location"})

ncvs['newcrime'] = ncvs['newcrime'].replace({"1":"Rape/sexual assault",
        "2": "Robbery",
        "3": "Aggravated assault",
        "4": "Simple assault",
        "5": "Personal theft"})

ncvs.head()

,region,file report,year,weight,ager,gender,hincome,newcrime,locationr
0,South,Yes,2014,2877.2646,25-34,Female,25k-34.9k,Rape/sexual assault,friend/neighbor/relative's home
1,South,No,2014,4079.9336,21-24,Male,Unknown,Aggravated assault,com/parking lot or public
2,South,Yes,2014,5984.119,21-24,Female,Unknown,Aggravated assault,friend/neighbor/relative's home
3,South,No,2014,3271.8376,35-49,Male,25k-34.9k,Aggravated assault,other location
4,South,Yes,2014,3402.2559,35-49,Male,Unknown,Simple assault,victim's home


In [201]:
# Set url for API
url = "https://api.bjs.ojp.gov/bjs/ncvs/v2/household/"

# Set empty lists to hold data
weight_h =[]
ager_h = []
gender_h = []
hincome_h = []
newcrime_h = []
locationr_h = []
years_h = []
region_h=[]
notify_h=[]


# Loop through each year
for year in year_list:

    # Create search query, make a request and store in JSON file
    query = url + year + "?format=json"
    response = requests.get(query)
    response_json = response.json()
    
    data_list=response_json["householdData"]
  
    for data in data_list:
        
        try: 
            region_h.append(data['region'])
            years_h.append(data['year'])
            weight_h.append(data['weight'])
            ager_h.append(data['hhage'])
            notify_h.append(data['notify'])
            gender_h.append(data['hhgen'])
            hincome_h.append(data['hincome'])
            newcrime_h.append(data['newoff'])
            locationr_h.append(data['locationr'])
        except Exception:
           print(data)


In [208]:
ncvs_h=pd.DataFrame({
    "region_h":region_h,
    "notify_h":notify_h,
    "year_h": years_h,
    "weight_h": weight_h,
    "age_h": ager_h,
    "gender_h": gender_h,
    "hincome_h": hincome_h,
    "newcrime_h": newcrime_h,
    "locationr_h": locationr_h})

In [211]:
ncvs_h.head()

,region_h,notify_h,year_h,weight_h,age_h,gender_h,hincome_h,newcrime_h,locationr_h
0,West,Yes,2014,2838.076,20-34,Female,25k-34.9k,Burglary/trespassing,victim's home
1,West,Yes,2014,3188.369,50-64,Female,75k+,Theft,com/parking lot or public
2,West,No,2014,3303.2144,35-49,Female,75k+,Theft,victim's home
3,West,Yes,2014,3795.8252,20-34,Male,25k-34.9k,Theft,victim's home
4,West,No,2014,3044.386,35-49,Male,<7.5k,Theft,victim's home


In [212]:
ncvs_h['age_h'] = ncvs_h['age_h'].replace({"1":"<19",
        "2":"20-34",
        "3": "35-49",
        "4": "50-64",
        "5": "65+"})

ncvs_h['gender_h'] = ncvs_h['gender_h'].replace({"1":"Male",
        "2":"Female"})


ncvs_h['region_h'] = ncvs_h['region_h'].replace({"1":"Northeast",
        "2": "Midwest",
        "3": "South",
        "4": "West"})

ncvs_h['notify_h'] = ncvs_h['notify_h'].replace({"1":"Yes",
        "2": "No",
        "3": "Do not know"})


ncvs_h['hincome_h'] = ncvs_h['hincome_h'].replace({"1":"<7.5k",
        "2": "7.5k-14.9k",
        "3": "15k-24.9k",
        "4": "25k-34.9k",
        "5": "35k-49.9k",
        "6": "50k-74.9k",
        "7": "75k+",
        "88": "Unknown"})

ncvs_h['locationr_h'] = ncvs_h['locationr_h'].replace({"1":"victim's home",
        "2": " friend/neighbor/relative's home",
        "3": "com/parking lot or public",
        "4": "school",
        "5": "other location"})

ncvs_h['newcrime_h'] = ncvs_h['newcrime_h'].replace({"8":"Theft",
        "7": "Motor-vehicle theft",
        "6": "Burglary/trespassing"})

In [213]:
ncvs_h.head()

,region_h,notify_h,year_h,weight_h,age_h,gender_h,hincome_h,newcrime_h,locationr_h
0,West,Yes,2014,2838.076,20-34,Female,25k-34.9k,Burglary/trespassing,victim's home
1,West,Yes,2014,3188.369,50-64,Female,75k+,Theft,com/parking lot or public
2,West,No,2014,3303.2144,35-49,Female,75k+,Theft,victim's home
3,West,Yes,2014,3795.8252,20-34,Male,25k-34.9k,Theft,victim's home
4,West,No,2014,3044.386,35-49,Male,<7.5k,Theft,victim's home
